# Homework
## Dataset
In this homework, we will continue the New York City Airbnb Open Data.

We'll keep working with the `price` variable, and we'll transform it to a classification task.

## Features
For the rest of the homework, you'll need to use the features from the previous homework with additional two `neighbourhood_group` and `room_type`. So the whole feature set will be set as follows:

* `neighbourhood_group`,
* `room_type`,
* `latitude`,
* `longitude`,
* `price`,
* `minimum_nights`,
* `number_of_reviews`,
* `reviews_per_month`,
* `calculated_host_listings_count`,
* `availability_365`

Select only them

In [1]:
import numpy as np
import pandas as pd

columns = [
    'neighbourhood_group',
    'room_type',
    'latitude',
    'longitude',
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'availability_365'    
]
df = pd.read_csv('AB_NYC_2019.csv')[columns]
df.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.64749,-73.97237,149,1,9,0.21,6,365
1,Manhattan,Entire home/apt,40.75362,-73.98377,225,1,45,0.38,2,355
2,Manhattan,Private room,40.80902,-73.94190,150,3,0,NaN,1,365
3,Brooklyn,Entire home/apt,40.68514,-73.95976,89,1,270,4.64,1,194
4,Manhattan,Entire home/apt,40.79851,-73.94399,80,10,9,0.10,1,0


## Question 1
What is the most frequent observation (mode) for the column `neighbourhood_group`?

In [2]:
df.groupby('neighbourhood_group').size()

neighbourhood_group
Bronx             1091
Brooklyn         20104
Manhattan        21661
Queens            5666
Staten Island      373
dtype: int64

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value (`'price'`) is not in your dataframe.

In [3]:
from sklearn.model_selection import train_test_split

def split_data(df, seed=42):
    train, test = train_test_split(df, test_size=0.2, random_state=seed)
    train, val = train_test_split(test, test_size=0.25, random_state=seed)
    
    dfs = [[d.drop('price', axis=1).reset_index(drop=True), d['price'].reset_index(drop=True)]
          for d in [train, val, test]]
    
    return [item for pair in dfs for item in pair]

In [4]:
trainX, trainy, valX, valy, testX, testy = split_data(df)
trainX.head()

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.62834,-73.90506,2,37,2.19,1,67
1,Manhattan,Entire home/apt,40.73476,-74.00100,1,7,0.12,1,0
2,Manhattan,Private room,40.85612,-73.93024,1,50,1.32,1,90
3,Manhattan,Private room,40.73083,-73.99430,4,0,NaN,1,0
4,Queens,Shared room,40.71540,-73.91148,1,5,2.68,1,89
